In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 32
block_size = 128
max_iters = 200
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

print(device)

cuda


C:\Users\Kenneth\AppData\Local\Temp\ipykernel_25144\1860295898.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pip list

Package                   Version
------------------------- ----------
anyio                     4.2.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.2.0
Babel                     2.14.0
beautifulsoup4            4.12.2
bleach                    6.1.0
Brotli                    1.0.9
certifi                   2023.11.17
cffi                      1.16.0
charset-normalizer        2.0.4
colorama                  0.4.6
comm                      0.2.1
cryptography              41.0.7
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.1
fastjsonschema            2.19.1
filelock                  3.13.1
fqdn                      1.5.1
fsspec                    2023.4.0
gmpy2                     2.1.2
idna                      3.4
ipykernel                 6.28.0
ipython          

In [3]:
chars = ""
with open(r"C:\Users\Kenneth\Desktop\python_projects\gpt-course\Large-Language-Model\vocab.txt", encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [10]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [ ]:
def get_random_chunk(split):
    filename = "openwebtext/train_split.txt" if split == 'train' else "openwebtext/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [30]:
class Head(nn.Module):
    """ This class represents one head of self-attention, a crucial component in 
    Transformer models for tasks such as natural language processing. """

    def __init__(self, head_size):
        '''They transform the input embeddings (n_embd) into keys, queries, and values for the attention mechanism.'''
        super().__init__() #  This defines the constructor method
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) 
        #This matrix is used to construct a causal mask ensuring that during self-attention, each position can only attend to positions before it in the sequence.
        self.dropout = nn.Dropout(dropout) # Dropout is a regularization technique used to prevent overfitting by randomly setting a fraction of input units to zero during training.

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape # Obtains the batch size (B), sequence length (T), and input embedding size (C) from the input tensor x.
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T) #  It performs a dot product between queries and keys, scaled by the square root of the key size.
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)  # Applies a mask to the attention scores to ensure causality, i.e., each position can only attend to positions before it in the sequence.
        wei = F.softmax(wei, dim=-1) # (B, T, T) #  Applies a softmax function to obtain the attention weights.
        wei = self.dropout(wei) #  Applies dropout to the attention weights for regularization.
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs) #  Computes the weighted sum of values using the attention weights.
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [38]:
import torch
import torch.nn as nn

# Define the input dimensions
n_embd = 128  # Input embedding size
head_size = 64  # Head size for linear transformation

# Create an instance of nn.Linear
linear_layer = nn.Linear(n_embd, head_size, bias=False)

# Generate some random input data
input_data = torch.randn(2, n_embd)  # Example batch size of 2

# Pass the input data through the linear layer
output_data = linear_layer(input_data)

# Display the output value
output_data

tensor([[ 0.8208,  0.0652,  0.0449, -0.3609,  0.8667, -0.2947, -0.2741,  0.1935,
         -0.1948, -0.6649, -0.6063, -0.6419, -0.8821, -0.1561,  0.7525, -0.8079,
          0.1865, -0.5883, -0.1600,  0.2688,  0.0817,  0.1296, -1.3025, -0.0581,
          0.5290,  0.1444, -0.4970,  1.2848, -1.0528,  0.0347, -0.2273,  0.2239,
          0.2425, -0.4471, -0.5768, -0.5690,  0.3356, -0.7686, -0.0229, -0.0316,
          0.0594,  0.0431, -0.2735, -0.6617, -0.3064,  0.8235, -0.6719, -0.7312,
         -0.3184,  0.2093, -0.6100,  0.0640, -0.5136, -0.7975, -0.3740, -0.5488,
         -0.0077,  0.2797,  0.0140,  0.4360, -0.1904,  0.7037,  0.4323, -0.0119],
        [-0.4265,  0.2315, -0.6117,  0.4909, -0.2933, -0.6605, -1.3260,  0.4108,
         -0.9564, -1.0663, -0.5276, -0.2177,  0.3193, -0.5281, -0.1108,  0.3351,
          0.4760, -0.6309,  0.8133, -0.0098, -1.2584,  0.2803,  0.6250,  0.2732,
         -0.2160, -0.8056, -0.2183,  0.2993, -0.6386,  1.0062,  0.5774, -0.4043,
          0.8505, -0.2440, 

In [31]:
# Test the Head class
head_size = 64  # Example head size
batch_size = 2
sequence_length = 10
embedding_size = 128

# Create an instance of the Head class
head = Head(head_size)

# Generate some random input data
input_data = torch.randn(batch_size, sequence_length, embedding_size)

# Pass the input data through the Head instance
output = head(input_data)

# Verify the shape of the output
print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 64])


In [46]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size): #  It takes two arguments: num_heads and head_size.
        super().__init__() #  This defines the constructor method
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # This creates a list of num_heads instances of the Head class. Each instance represents one head of self-attention.
        self.proj = nn.Linear(head_size * num_heads, n_embd) #  This defines a linear transformation layer (nn.Linear) that projects the concatenated outputs of the individual heads back to the original embedding dimension n_embd.
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3]) #  applies each head of self-attention (h(x)) to the input tensor x and concatenates the outputs along the last dimension (dim=-1)
        print(out.shape)
        out = self.dropout(self.proj(out))
        return out

In [48]:
import torch
import torch.nn as nn

# Create an instance of nn.Linear
linear_layer = nn.Linear(in_features=100, out_features=50)  # Input size = 100, Output size = 50

# Generate some random input data
input_data = torch.randn(32, 100)  # Batch size = 32, Number of input features = 100

# Pass the input data through the linear layer
output_data = linear_layer(input_data)

# Print the shape of the output tensor
print("Output shape:", output_data.shape)

Output shape: torch.Size([32, 50])


In [47]:
import torch
import torch.nn as nn

# Define the parameters (replace these with your actual values)
n_embd = 128
num_heads = 4
head_size = 32
dropout = 0.1

# Create an instance of the MultiHeadAttention class
multihead_attention = MultiHeadAttention(num_heads, head_size)

# Generate some random input data
batch_size = 2
sequence_length = 10
input_data = torch.randn(batch_size, sequence_length, n_embd)

# Pass the input data through the MultiHeadAttention instance
output = multihead_attention(input_data)

# Verify the shape of the output
print("Output shape:", output.shape)

torch.Size([2, 10, 128])
Output shape: torch.Size([2, 10, 128])


In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    

In [32]:
# To test head class

import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the parameters (replace these with your actual values)
n_embd = 128
head_size = 64
block_size = 10
dropout = 0.1

# Test the Head class
batch_size = 2
sequence_length = 10

# Create an instance of the Head class
head = Head(head_size)

# Generate some random input data
input_data = torch.randn(batch_size, sequence_length, n_embd)

# Pass the input data through the Head instance
output = head(input_data)

# Verify the shape of the output
print("Output shape:", output.shape)

Output shape: torch.Size([2, 10, 64])


In [26]:
k = nn.Linear(128, 64, bias=False)
print(k)

Linear(in_features=128, out_features=64, bias=False)


In [27]:
l = k.transpose(-2, -1)
l.shape

AttributeError: 'Linear' object has no attribute 'transpose'